In [1]:
!pip install yfinance

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 52.2 MB/s eta 0:00:0000:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.0/73.0 kB 4.2 MB/s eta 0:00:00
  Created wheel for peewee: filename=peewee-3.17.1-cp310-cp310-linux_x86_64.whl size=291653 sha256=d903b9f002932172a19bcade47fe149ed133409a491e7281af75e464daafbb53
  Stored in directory: /root/.cache/pip/wheels/d7/35/5c/1374782be033462df5f40174d8d879519d64ed8c25a1977554
Successfully built peewee


In [2]:
# Create a directory if it is not there, so we can save files and results in it
from pathlib import Path
Path('/kaggle/working/PSO').mkdir(parents=True,exist_ok=True)

In [5]:
# -*- coding: utf-8 -*-
"""notebooka94414a4a1

Automatically generated by Colaboratory.

Original file is located at
    https://colab.research.google.com/#fileId=https%3A//storage.googleapis.com/kaggle-colab-exported-notebooks/notebooka94414a4a1-780af299-0af7-47f5-8d22-fbda06b2cfac.ipynb%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com/20240409/auto/storage/goog4_request%26X-Goog-Date%3D20240409T060240Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D2a380db4b5c7c975d672260e4a1cfddac2ca862101743509ae673b4ce1adf614a57c12f2035a06d4c45d0dd4c364e44d77dffb9ed4366cc9e9eec3f9c4e3168cf60be9d9e2b9c8aae18437720530b663922c05aaf84f422131cd95346b33b45448998deedc542a2de88805b3d53237d6bddaf32b9d7dc27a07613a58f86577111558ac04060ff48e761fc184e6f7db48697c83031fb9ced6fbc4146acc6236fd740df8eba740c27f77c04a92385a642eeb3a3f00c0a44e398be6f81448a60edb1f70c3d138aab39759b06fa26e8fbc755ef6ac06202b26c4bf4a345d7c16b1346f644b62267e987ce8ae94d7cd377d19eea834ca86c9abe523c6284541454ebe
"""

import numpy as np
import pandas as pd
import yfinance as yf
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error, r2_score

def stock_predict(stockname,filename):

    # Download stock data
    symbol = stockname
    start_date = '2019-01-01'
    end_date = '2024-01-01'
    stock_data = yf.download(symbol, start=start_date, end=end_date)
    specific_df = pd.DataFrame(stock_data).reset_index()

    # Preprocess stock data
    specific_df['Date'] = pd.to_datetime(specific_df['Date'])
    specific_df['Year'] = specific_df['Date'].dt.year
    specific_df['Month'] = specific_df['Date'].dt.month
    specific_df['Day'] = specific_df['Date'].dt.day

    # Normalize data
    scaler = MinMaxScaler()
    scaled_data = scaler.fit_transform(specific_df[['Close']])

    # Split data into training and testing sets
    train_size = int(len(scaled_data) * 0.8)
    train_data, test_data = scaled_data[0:train_size, :], scaled_data[train_size:len(scaled_data), :]

    # Generate sequences and labels for LSTM
    def generate_sequences_and_labels(data, n_past):
        sequences, labels = [], []
        for i in range(len(data) - n_past):
            sequences.append(data[i:i + n_past])
            labels.append(data[i + n_past, 0])
        return np.array(sequences), np.array(labels)

    n_past = 2
    x_train, y_train = generate_sequences_and_labels(train_data, n_past)
    x_test, y_test = generate_sequences_and_labels(test_data, n_past)
    x_train_lstm = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1))
    x_test_lstm = np.reshape(x_test, (x_test.shape[0], x_test.shape[1], 1))
    
    ################################ START ###########################################

    def objective_function(params, X_train, Y_train, X_test, Y_test):
        lstm_units, dropout_rate, dense_units = params

        model = Sequential()
        model.add(LSTM(units=int(lstm_units), return_sequences=True, input_shape=(n_past, 1)))
        model.add(Dropout(dropout_rate))
        model.add(LSTM(units=int(lstm_units/2), return_sequences=False))
        model.add(Dropout(dropout_rate))
        model.add(Dense(int(dense_units), activation='relu'))
        model.add(Dense(1))

        model.compile(optimizer='adam', loss='mean_squared_error')
        model.fit(X_train, Y_train, epochs=100, verbose=0)

        predictions = model.predict(X_test)
        r2 = r2_score(Y_test, predictions)
        #mse = mean_squared_error(Y_test, predictions)
        return -r2  # negative R-squared for minimization

    def particle_swarm_optimization(objective_function, no_particle, no_dim, x_range, v_range, iw_range, c, X_train, Y_train, X_test, Y_test, print_step, iter):

        particles = np.random.uniform([r[0] for r in x_range], [r[1] for r in x_range], size=(no_particle, no_dim))
        velocities = np.random.uniform([r[0] for r in v_range], [r[1] for r in v_range], size=(no_particle, no_dim))
        pbest = np.full(no_particle, np.inf)
        pbestpos = np.zeros((no_particle, no_dim))
        gbest = np.inf
        gbestpos = np.zeros((no_dim,))

        for i in range(iter):

            for j, particle in enumerate(particles):

                fitness = objective_function(particle, X_train, Y_train, X_test, Y_test)

                if fitness < pbest[j]:
                    pbest[j] = fitness
                    pbestpos[j] = particle.copy()
                if fitness < gbest:
                    gbest = fitness
                    gbestpos = particle.copy()

            for j, particle in enumerate(particles):
                iw = np.random.uniform(iw_range[0], iw_range[1], 1)[0]
                velocities[j] = (
                    iw * velocities[j]
                    + (c[0] * np.random.uniform(0.0, 1.0, (no_dim,)) * (pbestpos[j] - particle))
                    + (c[1] * np.random.uniform(0.0, 1.0, (no_dim,)) * (gbestpos - particle))
                )
                particles[j] += velocities[j]

        return gbestpos

    # Run PSO optimization
    no_particle = 10
    no_dim = 3
    x_range = [(50, 200), (0.1, 0.5), (8, 32)]
    v_range = [(1, 10), (0.01, 0.1), (1, 10)]
    iw_range = (0.1, 0.5)
    c = (1.4962, 1.4962)  # cognitive and social parameters
    # Run PSO optimization
    best_params = particle_swarm_optimization(objective_function, no_particle, no_dim, x_range, v_range, iw_range, c, x_train, y_train, x_test, y_test, 10, 2)

    # Define the ranges for LSTM parameters
    lstm_units, dropout_rate, dense_units = best_params

    # Define the create the main lstm model for final prediction

    def lstm_Model(lstm_units, dropout_rate, dense_units, n_past):
        lstm_model = Sequential()
        lstm_model.add(LSTM(units=int(lstm_units), return_sequences=True, input_shape=(n_past, 1)))
        lstm_model.add(Dropout(dropout_rate))
        lstm_model.add(LSTM(units=int(lstm_units/2), return_sequences=False))
        lstm_model.add(Dropout(dropout_rate))
        lstm_model.add(Dense(int(dense_units), activation='relu'))
        lstm_model.add(Dense(1))
        lstm_model.compile(optimizer='adam', loss='mean_squared_error')

        return lstm_model

    # Train the LSTM model
    lstm_model = lstm_Model(lstm_units, dropout_rate, dense_units, n_past)
    history = lstm_model.fit(x_train_lstm, y_train, epochs=200, validation_data=(x_test_lstm, y_test), verbose=2)
    
    ######################################### END ###########################################
    
    folder_path = "/kaggle/working/PSO/"  # Get the current script's directory
    filename_a = folder_path+filename+'_PSO.keras'
    lstm_model.save(filename_a)

#     model = tf.keras.models.load_model(filename_a)
    
    # Plotting the learning curve

    predictions = lstm_model.predict(x_test)
    # Inverse transform the predicted and actual prices
    predicted_prices = scaler.inverse_transform(predictions)
    actual_prices = scaler.inverse_transform(y_test.reshape(-1, 1))

    # Calculate evaluation metrics
    r2 = r2_score(actual_prices, predicted_prices)
    mae = mean_absolute_error(actual_prices, predicted_prices)
    mse = mean_squared_error(actual_prices, predicted_prices)
    mape = np.mean(np.abs((actual_prices - predicted_prices) / actual_prices)) * 100

    # Print evaluation metrics
    print("R-squared:", r2)
    print("MAE:", mae)
    print("MSE:", mse)
    print("MAPE:", mape)

    # Plot actual vs predicted prices
    plt.plot(actual_prices, label='Actual')
    plt.plot(predicted_prices, label='Predicted')
    plt.title(stockname+'-PSO')
#     plt.title('Stock Price Prediction using LSTM+PSO')
    plt.xlabel('Time')
    plt.ylabel('Stock Price')
    plt.legend()
    plt_name = folder_path+filename+'_PSO.png'
    plt.savefig(plt_name)
    plt.show()

In [6]:
stock_list = ["AAPL","AMGN","AXP","BA","CAT","CRM","CSCO","CVX","DIS","DOW","GS","HD","HON",
              "IBM","INTC","JNJ","JPM","KO","MCD","MMM","MRK","MSFT","NKE","PG","TRV","UNH",
              "V","VZ","WBA","WMT"]
print(len(stock_list))

30


In [ ]:
for stock in stock_list:
    stock_predict(stock,stock)

[*********************100%%**********************]  1 of 1 completed
/opt/conda/lib/python3.10/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step


/opt/conda/lib/python3.10/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step


/opt/conda/lib/python3.10/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


...

8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step


/opt/conda/lib/python3.10/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step


/opt/conda/lib/python3.10/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step


In [ ]:
!cd /kaggle/working
!zip -r PSO.zip /kaggle/working/PSO  